# 1. Prepare Datasets

## Import the S3 data into SageMaker

In [2]:
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


## Store S3 locations

In [3]:
s3_public_path_kaggle = "s3://collegeaffordability317/Kaggle/"
s3_public_path_tuition = "s3://collegeaffordability317/TuitionTracker/"
s3_public_path_usda = "s3://collegeaffordability317/USDA/"
s3_public_path_usde = "s3://collegeaffordability317/USDE/"

In [4]:
%store s3_public_path_kaggle
%store s3_public_path_tuition
%store s3_public_path_usda
%store s3_public_path_usde

Stored 's3_public_path_kaggle' (str)
Stored 's3_public_path_tuition' (str)
Stored 's3_public_path_usda' (str)
Stored 's3_public_path_usde' (str)


## Show all the data files for the project

In [5]:
!aws s3 ls $s3_public_path_kaggle

2024-03-17 16:07:03   15143141 college_data.csv


In [6]:
!aws s3 ls $s3_public_path_tuition

2024-03-17 16:07:00      21527 DataDictionary.xlsx
2024-03-17 16:07:00     838246 GradRates.csv
2024-03-17 16:07:01    1319314 NetPrice.csv
2024-03-17 16:07:01    1382164 RetentionRates.csv
2024-03-17 16:07:02     653000 StickerPrice.csv


In [7]:
!aws s3 ls $s3_public_path_usda

2024-03-17 16:06:57    1610608 Education.xlsx
2024-03-17 16:06:58    1047684 PopulationEstimates.xlsx
2024-03-17 16:06:58     604165 PovertyEstimates.xlsx
2024-03-17 16:06:59    2762055 Unemployment.xlsx


In [8]:
!aws s3 ls $s3_public_path_usde

2024-03-17 16:06:52      59904 FedStudentAidPortfolioByAge.xls
2024-03-17 16:06:53      68096 FedStudentAidPortfolioByDebtSize.xls
2024-03-17 16:06:53      62976 FedStudentAidPortfolioByLoanType.xls
2024-03-17 16:06:53      50176 FedStudentAidPortfolioByLocation.xls
2024-03-17 16:06:54      60928 FedStudentAidPortfolioBySchoolType.xls
2024-03-17 16:06:54      61952 FedStudentAidPortfolioSummary.xls
2024-03-17 16:06:55      52224 PortfolioByAgeAndDebtSize.xls
2024-03-17 16:06:55      91648 PortfolioByDelinquencyStatus.xls
2024-03-17 16:06:56      57856 PortfolioByLocationAndAge.xls
2024-03-17 16:06:56      58368 PortfolioByLocationAndDebtSize.xls
2024-03-17 16:06:56     147456 PortfolioByRepaymentPlan.xls


# 2. Athena DB

In [11]:
from pyathena import connect

# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [12]:
# Set Athena parameters
database_name = 'collegeaffordability317'
table_name = 'college_data'

In [13]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [14]:
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)

pd.read_sql(statement, conn)

""


In [15]:
statement = "SHOW DATABASES"

df_show = pd.read_sql(statement, conn)
df_show.head(10)

,database_name
0,college_afford
1,collegeaff
2,collegeaffordability317
3,default
4,dsoaws


In [16]:
# Drop the table if it already exists
pd.read_sql(f'DROP TABLE IF EXISTS {database_name}.{table_name}', conn)

# Define the CREATE TABLE statement with data types in lowercase
statement = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {database_name}.{table_name} (
    unitid int,
    instnm string,
    city string,
    stabbr string,
    zip string,
    region string,
    preddeg string,
    locale string,
    latitude float,
    longitude float,
    ccbasic string,
    ccugprof string,
    ccsizset string,
    adm_rate_all float,
    pptug_ef float,
    sat_avg_all float,
    costt4_a float,
    control string,
    tuitionfee_in float,
    tuitionfee_out float,
    median_hh_inc float,
    mn_earn_wne_indep0_p10 float,
    mn_earn_wne_indep1_p10 float,
    ugds_white float,
    ugds_black float,
    ugds_hisp float,
    ugds_asian float,
    ugds_aian float,
    ugds_nhpi float,
    ugds_2mor float,
    ugds_nra float,
    ugds_unkn float,
    year int
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
LOCATION 's3://collegeaffordability317/Kaggle/'
"""

In [17]:
# Execute the CREATE TABLE statement
table_1 = pd.read_sql(statement, conn)

In [18]:
# Drop the table if it already exists
pd.read_sql(f'DROP TABLE IF EXISTS {database_name}.{table_name}', conn)

# Define the CREATE TABLE statement with data types in lowercase
statement = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {database_name}.{table_name} (
    unitid int,
    inst_nm string,
    nonres_alien_4yr int,
    unkown_ethn_4yr int,
    grand_total_4yr int,
    hispanic_4yr int,
    amindian_alaskanative_4yr int,
    asian_4yr int,
    black_african_4yr int,
    native_hawaiian_4yr int,
    unknown_ethn_2yr int, 
    grand_total_2_yr int,
    hispanic_2yr int,
    amindian_alaskanative_2yr int,
    asian_2yr int,
    black_african_2yr int,
    native_hawaiian_2yr int,
    white_2yr int,
    two_or_more_race_2yr int,
    unnamed float
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
LOCATION 's3://collegeaffordability317/TuitionTracker/'
"""

In [19]:
# Execute the CREATE TABLE statement
table_2 = pd.read_sql(statement, conn)

# 2. Exploratory Data Analysis

In [20]:
import csv

df_colleges = pd.read_csv(s3_public_path_kaggle + 
                          "college_data.csv")
df_colleges.shape

/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3508: DtypeWarning: Columns (10,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(58123, 33)

In [21]:
df_gradrate = pd.read_csv(s3_public_path_tuition + 
                          "GradRates.csv")
df_netprice = pd.read_csv(s3_public_path_tuition + 
                          "NetPrice.csv")
df_retentionrate = pd.read_csv(s3_public_path_tuition + 
                          "RetentionRates.csv")
df_stickerprice = pd.read_csv(s3_public_path_tuition + 
                          "StickerPrice.csv")

In [22]:
df_stickerprice.head()

,UnitID,Institution Name,Sector of institution (HD2021),Total price for in-state students living on campus 2021-22 (DRVIC2021),Total price for in-state students living off campus (not with family) 2021-22 (DRVIC2021),Total price for in-state students living off campus (with family) 2021-22 (DRVIC2021),Total price for in-state students living on campus 2020-21 (DRVIC2020),Total price for in-state students living off campus (not with family) 2020-21 (DRVIC2020),Total price for in-state students living off campus (with family) 2020-21 (DRVIC2020),Total price for in-state students living on campus 2019-20 (DRVIC2019),...,Total price for in-state students living off campus (with family) 2014-15 (DRVIC2014),Total price for in-state students living on campus 2013-14 (DRVIC2013_RV),Total price for in-state students living off campus (not with family) 2013-14 (DRVIC2013_RV),Total price for in-state students living off campus (with family) 2013-14 (DRVIC2013_RV),Total price for in-state students living on campus 2012-13 (DRVIC2012_RV),Total price for in-state students living off campus (not with family) 2012-13 (DRVIC2012_RV),Total price for in-state students living off campus (with family) 2012-13 (DRVIC2012_RV),Total price for in-state students living on campus 2011-12 (DRVIC2011_RV),Total price for in-state students living off campus (not with family) 2011-12 (DRVIC2011_RV),Total price for in-state students living off campus (with family) 2011-12 (DRVIC2011_RV)
0,180203,Aaniiih Nakoda College,1,NaN,17030.0,8510.0,NaN,17030.0,8510.0,NaN,...,8510.0,NaN,17030.0,8510.0,NaN,17030.0,8510.0,NaN,17030.0,8510.0
1,222178,Abilene Christian University,2,55500.0,55500.0,43872.0,53672.0,53672.0,42322.0,51887.0,...,34100.0,41800.0,41800.0,33000.0,39900.0,39900.0,31250.0,38250.0,NaN,NaN
2,497037,Abilene Christian University-Undergraduate Online,2,NaN,30670.0,19042.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,138558,Abraham Baldwin Agricultural College,1,15727.0,13965.0,7765.0,15575.0,13865.0,7665.0,15479.0,...,6894.0,17503.0,13188.0,7578.0,16550.0,12619.0,7009.0,12347.0,NaN,NaN
4,488031,Abraham Lincoln University,3,NaN,27133.0,11365.0,NaN,25576.0,11176.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Release SageMaker Resources

In [23]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}